In [1]:
!pip install pyspark==3.4.0
!pip install findspark

  Using cached pyspark-3.4.0.tar.gz (310.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317159 sha256=d055d6199aa5b5887b95168f5aab42eea4c6935d4d2c5be401f1131cd7d764ed
  Stored in directory: c:\users\test\appdata\local\pip\cache\wheels\9f\34\a4\159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import findspark
print(findspark.init())

None


In [11]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


NameError: name 'pyspark' is not defined

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()




In [4]:
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession is active and ready to use.")
else:
    print("SparkSession is not active. Please create a SparkSession.")

SparkSession is active and ready to use.


In [5]:
data = range(1,30)
# print first element of iterator
len(data)
xrangeRDD = sc.parallelize(data, 4)

# this will let us know that we created an RDD
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<10)
filteredRDD

PythonRDD[1] at RDD at PythonRDD.scala:53

In [ ]:
!python3 -m pip install pyspark==3.4.0

In [6]:
# import os,sys
# os.environ["PYSPARK_PYTHON"] = r"C:\Users\TEST\AppData\Local\Programs\Python\Python39\python3.exe"
# os.environ["PYSPARK_DRIVER_PYTHON "] = r"C:\Users\TEST\AppData\Local\Programs\Python\Python39\python3.exe"

print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


10

In [13]:
spark.stop()

In [23]:
# Narrow Transformation
## Map

from pyspark import SparkContext
sc = SparkContext("local", "MapExample")
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
# mapped_rdd = rdd.map(lambda x: x * 2)
# mapped_rdd.collect() # Output: [2, 4, 6, 8, 10]

filtered_rdd = rdd.filter(lambda x: x % 2 == 0)
filtered_rdd.collect() # Output: [2, 4]




[2, 4]

In [28]:
sc.stop()

In [25]:
from pyspark import SparkContext
sc = SparkContext("local", "UnionExample")
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
union_rdd = rdd1.union(rdd2)
union_rdd.collect() # Output: [1, 2, 3, 4, 5, 6]

[1, 2, 3, 4, 5, 6]

In [29]:
## Wide Transformation

from pyspark import SparkContext
sc = SparkContext("local", "GroupByExample")
data = [("apple", 2), ("banana", 3), ("apple", 5), ("banana", 1)]
rdd = sc.parallelize(data)
grouped_rdd = rdd.groupBy(lambda x: x[0])
sum_rdd = grouped_rdd.mapValues(lambda values: sum([v[1] for v in values]))
sum_rdd.collect() # Output: [('apple', 7), ('banana', 4)]

[('apple', 7), ('banana', 4)]

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("RuleBasedTransformations").getOrCreate()

# Sample input data for DataFrame 1
data1 = [("Alice", 25, "F"), ("Bob", 30, "M"), ("Charlie", 22, "M"), ("Diana", 28, "F")]

# Sample input data for DataFrame 2
data2 = [("Alice", "New York"), ("Bob", "San Francisco"), ("Charlie", "Los Angeles"), ("Eve", "Chicago")]

# Create DataFrames
columns1 = ["name", "age", "gender"]
df1 = spark.createDataFrame(data1, columns1)
columns2 = ["name", "city"]
df2 = spark.createDataFrame(data2, columns2)

# Applying Predicate Pushdown (Filtering)
filtered_df = df1.filter(col("age") > 25)

# Applying Constant Folding
folded_df = filtered_df.select(col("name"), col("age") + 2)

# Applying Column Pruning
pruned_df = folded_df.select(col("name"))

# Join Reordering
reordered_join = df1.join(df2, on="name")

# Show the final results
print("Filtered DataFrame:")
filtered_df.show()
print("Folded DataFrame:")
folded_df.show()
print("Pruned DataFrame:")
pruned_df.show()
print("Reordered Join DataFrame:")
reordered_join.show()

# Stop the Spark session
spark.stop()

Filtered DataFrame:
+-----+---+------+
| name|age|gender|
+-----+---+------+
|  Bob| 30|     M|
|Diana| 28|     F|
+-----+---+------+

Folded DataFrame:
+-----+---------+
| name|(age + 2)|
+-----+---------+
|  Bob|       32|
|Diana|       30|
+-----+---------+

Pruned DataFrame:
+-----+
| name|
+-----+
|  Bob|
|Diana|
+-----+

Reordered Join DataFrame:
+-------+---+------+-------------+
|   name|age|gender|         city|
+-------+---+------+-------------+
|  Alice| 25|     F|     New York|
|    Bob| 30|     M|San Francisco|
|Charlie| 22|     M|  Los Angeles|
+-------+---+------+-------------+

